# Notebook alternativo al Docker y SQL automatizado:

Este notebook funciona como una alternativa a la prepración de los datos para utilizarlos en la plataforma.

---

## Funcionamiento de la herramienta:

La herramienta esta con el dataset previamente cargado y con una ruta establecida hacia una carpeta de drive por lo que NO se debe de realizar ninguna modificación sobre el mismo simplemente correr el código.

---

## Construcción del código:

Importación de librerias

In [8]:
import pandas as pd
import numpy as np
import pickle
import datetime
import gdown

Se hace una conexión con drive y se seleeciona directamente el dataset comentado anteriormente:
*  HI-Small_Trans.csv

A su vez, se carga el modelo guardado en un archivo .pkl, llamado
*  model_xgb_optimized.pkl

In [9]:
# Descargar el dataset utilizando gdown
file_id = '1fDw0plQy898cw5aPZ8qBfP61djO6ssDH'
gdown.download(f'https://drive.google.com/uc?export=download&id={file_id}', 'hi_small.csv', quiet=False)

# Descargar el modelo utilizando gdown
model_file_id = '1p593_b7-TkBAFzfOoEuXfg3jsDD1jyjw'
gdown.download(f'https://drive.google.com/uc?export=download&id={model_file_id}', 'model_xgb_optimized.pkl', quiet=False)

# Descargar los label encoders utilizando gdown
encoders_file_id = '1fi9D8-UYu541eiHGDWugnat1Vavd9a3h'
gdown.download(f'https://drive.google.com/uc?export=download&id={encoders_file_id}', 'label_encoders1.pkl', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1fDw0plQy898cw5aPZ8qBfP61djO6ssDH
From (redirected): https://drive.google.com/uc?export=download&id=1fDw0plQy898cw5aPZ8qBfP61djO6ssDH&confirm=t&uuid=c9b59b40-acc6-48a3-94e6-10b8511a1a35
To: /content/hi_small.csv
100%|██████████| 476M/476M [00:09<00:00, 49.9MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1p593_b7-TkBAFzfOoEuXfg3jsDD1jyjw
To: /content/model_xgb_optimized.pkl
100%|██████████| 923k/923k [00:00<00:00, 91.2MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1fi9D8-UYu541eiHGDWugnat1Vavd9a3h
To: /content/label_encoders.pkl
100%|██████████| 71.7M/71.7M [00:00<00:00, 76.1MB/s]


'label_encoders.pkl'

En esta parte, preparamos los datos aplicandole los label encorders al dataset.

 Tanto a los positivos como a los negativos, posteriormente los añadimos en el mismo dataset para crear uno único. En este único, aplicamos las predicciones y las reglas SQL predefinidas. Una vez que las tenemos, utilizamos un sistema de prioridades pra los analistas, quedando asi:

 El sistema de prioridades quedaría así:

*   Prioridad 1: SQL + modelo             1/1
*   Prioridad 2:  Solo SQL detecta        1/0
*   Prioridad 3: Solo detecta modelo    0/1





In [10]:

# Cargar el dataset y el modelo
hi_small = pd.read_csv('hi_small.csv')

modelo_path = 'model_xgb_optimized.pkl'
with open(modelo_path, 'rb') as file:
    model = pickle.load(file)

# Preparación de datos para aplicar label encoders
hi_small['Timestamp'] = pd.to_datetime(hi_small['Timestamp'])

hi_small['Year'] = hi_small['Timestamp'].dt.year
hi_small['Month'] = hi_small['Timestamp'].dt.month
hi_small['Day'] = hi_small['Timestamp'].dt.day
hi_small['Hour'] = hi_small['Timestamp'].dt.hour
hi_small['Minute'] = hi_small['Timestamp'].dt.minute

hi_small.drop(columns=['Timestamp'], inplace=True)
hi_small.rename(columns={'Account': 'Account2', 'Account.1': 'Account4'}, inplace=True)

# Cargar los label encoders
path_to_encoders = 'label_encoders1.pkl'
with open(path_to_encoders, 'rb') as f:
    label_encoders = pickle.load(f)

# Aplicación de los labels encoders
for col, le in label_encoders.items():
    if col in hi_small.columns:
        hi_small[col] = le.transform(hi_small[col])

X = hi_small.drop(columns=['Is Laundering'])
predicciones = model.predict(X)

# Filtro de predicciones de 1
hi_small['predicciones'] = predicciones
hi_small_pred_1 = hi_small[hi_small['predicciones'] == 1]

# Aplicación inversa de predicciones = 1
with open(path_to_encoders, 'rb') as f:
    label_encoders = pickle.load(f)
for col, le in label_encoders.items():
    if col in hi_small_pred_1.columns:
        hi_small_pred_1[col] = le.inverse_transform(hi_small_pred_1[col])

hi_small_pred_1.rename(columns={'Account2': 'from_account', 'Account4': 'to_account', 'Is Laundering': 'Actual_Is_Laundering', 'predicciones': 'Predicted_Is_Laundering'}, inplace=True)
hi_small_pred_1.drop(columns=['Year', 'Month', 'Day', 'Hour', 'Minute'], inplace=True)

# Filtro de predicciones de 0
hi_small_pred_0 = hi_small[hi_small['predicciones'] == 0]
hi_small_pred_0_16000 = hi_small_pred_0.head(16000)

# Aplicación inversa de predicciones = 0
with open(path_to_encoders, 'rb') as f:
    label_encoders = pickle.load(f)
for col, le in label_encoders.items():
    if col in hi_small_pred_0_16000.columns:
        hi_small_pred_0_16000[col] = le.inverse_transform(hi_small_pred_0_16000[col])

hi_small_pred_0_16000.rename(columns={'Account2': 'from_account', 'Account4': 'to_account', 'Is Laundering': 'Actual_Is_Laundering', 'predicciones': 'Predicted_Is_Laundering'}, inplace=True)
hi_small_pred_0_16000.drop(columns=['Year', 'Month', 'Day', 'Hour', 'Minute'], inplace=True)

# Combinación de los dataset
combined_df = pd.concat([hi_small_pred_1, hi_small_pred_0_16000])
combined_df.columns = combined_df.columns.str.strip().str.replace(' ', '_').str.lower()
transactions = combined_df.drop("actual_is_laundering", axis=1)

# Reglas de SQL
def apply_aml_rules(df):
    rules = [
        {
            "name": "Cash Reporting Circumvention",
            "condition": "(payment_currency in ['Euro', 'US Dollar']) and (payment_format == 'Cash') and (9800 <= amount_paid <= 9999.99)"
        },
        {
            "name": "Exotic Currencies to Cash",
            "condition": "(payment_currency in ['Shekel', 'Mexican Peso', 'Brazilian Real']) and (payment_format in ['Cheque', 'Cash', 'Wire']) and (amount_paid >= 90000)"
        },
        {
            "name": "High Value Bitcoin to/from Cash",
            "condition": "((receiving_currency == 'Bitcoin') or (payment_currency == 'Bitcoin')) and (amount_paid > 35000)"
        },
        {
            "name": "Exotic Currencies to/from Bitcoin",
            "condition": "(((receiving_currency in ['Mexican Peso', 'Ruble']) and (payment_currency == 'Bitcoin')) or ((receiving_currency == 'Bitcoin') and (payment_currency in ['Mexican Peso', 'Ruble']))) and (amount_paid > 150000)"
        },
        {
            "name": "High Value Cheque Payments",
            "condition": "(payment_format == 'Cheque') and (amount_paid > 1000000)"
        },
        {
            "name": "High Value Wire Payments",
            "condition": "(payment_format == 'Wire') and (amount_paid > 5000000)"
        }
    ]

    df['sql_is_laundering'] = 0

    for rule in rules:
        df.loc[df.eval(rule['condition']), 'sql_is_laundering'] = 1

    return df

# Aplicación de las reglas sobre el dataset
transactions = apply_aml_rules(transactions)

# Asignación de las prioridades
def assign_priority(row):
    if row['predicted_is_laundering'] == 1 and row['sql_is_laundering'] == 0:
        return 'Medium Priority'
    elif row['predicted_is_laundering'] == 0 and row['sql_is_laundering'] == 1:
        return 'Normal Priority'
    elif row['predicted_is_laundering'] == 1 and row['sql_is_laundering'] == 1:
        return 'High Priority'
    else:
        return 'Low Priority'

transactions['priority'] = transactions.apply(assign_priority, axis=1)
transactions_filtered = transactions[(transactions['predicted_is_laundering'] != 0) | (transactions['sql_is_laundering'] != 0)]

# Exportar el JSON al local
output_json_filename = f'filtered_transactions_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.json'
transactions_filtered.to_json(output_json_filename, orient='records', lines=False)

# Descargar el archivo JSON al local
from google.colab import files
files.download(output_json_filename)

# Confirmación
print(f"El archivo JSON '{output_json_filename}' ha sido generado y descargado.")


<ipython-input-10-0b9dcdcb6d0c>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hi_small_pred_1[col] = le.inverse_transform(hi_small_pred_1[col])
<ipython-input-10-0b9dcdcb6d0c>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hi_small_pred_1[col] = le.inverse_transform(hi_small_pred_1[col])
<ipython-input-10-0b9dcdcb6d0c>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

El archivo JSON 'filtered_transactions_20240919_100950.json' ha sido generado y descargado.


In [5]:
# Cargar el dataset y el modelo
hi_small = pd.read_csv('hi_small.csv')

In [6]:
print(hi_small.head())  # Para ver las primeras filas del DataFrame
print(hi_small.columns)  # Para ver todas las columnas disponibles


Empty DataFrame
Columns: [<!DOCTYPE html><html><head><title>Google Drive - Virus scan warning</title><meta http-equiv="content-type" content="text/html; charset=utf-8"/><style nonce="80VdaYj7Y_pRnLKDv-DRxg">.goog-link-button{position:relative;color:#15c;text-decoration:underline;cursor:pointer}.goog-link-button-disabled{color:#ccc;text-decoration:none;cursor:default}body{color:#222;font:normal 13px/1.4 arial, sans-serif;margin:0}.grecaptcha-badge{visibility:hidden}.uc-main{padding-top:50px;text-align:center}#uc-dl-icon{display:inline-block;margin-top:16px;padding-right:1em;vertical-align:top}#uc-text{display:inline-block;max-width:68ex;text-align:left}.uc-error-caption, .uc-warning-caption{color:#222;font-size:16px}#uc-download-link{text-decoration:none}.uc-name-size a{color:#15c;text-decoration:none}.uc-name-size a:visited{color:#61c;text-decoration:none}.uc-name-size a:active{color:#d14836;text-decoration:none}.uc-footer{color:#777;font-size:11px;padding-bottom:5ex;padding-top:5ex;te

Finalmente, se guardan los resultados en formato JSON para poder utilizarlos correctamente en la plataforma

In [ ]:
# Filter 50% of Medium Priority and 50% of Normal Priority
medium_priority_df = transactions_filtered[transactions_filtered['priority'] == 'Medium Priority']
normal_priority_df = transactions_filtered[transactions_filtered['priority'] == 'Normal Priority']

# Ensure that there are at least 450 transactions in each category, if not, take all available
medium_priority_sample = medium_priority_df.sample(n=min(450, len(medium_priority_df)), random_state=42)
normal_priority_sample = normal_priority_df.sample(n=min(450, len(normal_priority_df)), random_state=42)

# Combine the samples
final_sample_df = pd.concat([medium_priority_sample, normal_priority_sample])

# Check if we have 900 records; if less, adjust or log it
if len(final_sample_df) < 900:
    print(f"Warning: The combined dataset has {len(final_sample_df)} records instead of 900 due to insufficient data.")

# Export the smaller dataset to a new JSON file
output_json_sample_path = f'/content/drive/MyDrive/TFM - Entregable/smaller_filtered_transactions_{current_time}.json'

# Save the smaller dataset
final_sample_df.to_json(output_json_sample_path, orient='records', lines=False)

# Print confirmation
print(f"Dataset with exported to {output_json_sample_path}")


Smaller dataset with 900 transactions exported to /content/drive/MyDrive/TFM - Entregable/smaller_filtered_transactions_20240918_235425.json
